In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import joblib
import os

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score,
    confusion_matrix, ConfusionMatrixDisplay
)
from sklearn.inspection import permutation_importance

# 0. Подготовка структуры папок
os.makedirs('artifacts/figures', exist_ok=True)


### 1. Загрузка данных и первичный анализ (EDA)

In [24]:
df = pd.read_csv('S06-hw-dataset-03.csv')

print("--- Базовый анализ ---")
print(f"Формат данных: {df.shape}")
print(df.info())

# Проверка баланса классов
target_balance = df['target'].value_counts(normalize=True)
print("\nБаланс классов:\n", target_balance)

# Сохранение графика баланса классов
plt.figure(figsize=(8, 5))
sns.countplot(x='target', data=df, palette='viridis')
plt.title('Распределение целевой переменной')
plt.savefig('artifacts/figures/target_balance.png')
plt.close()


--- Базовый анализ ---
Формат данных: (15000, 30)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      15000 non-null  int64  
 1   f01     15000 non-null  float64
 2   f02     15000 non-null  float64
 3   f03     15000 non-null  float64
 4   f04     15000 non-null  float64
 5   f05     15000 non-null  float64
 6   f06     15000 non-null  float64
 7   f07     15000 non-null  float64
 8   f08     15000 non-null  float64
 9   f09     15000 non-null  float64
 10  f10     15000 non-null  float64
 11  f11     15000 non-null  float64
 12  f12     15000 non-null  float64
 13  f13     15000 non-null  float64
 14  f14     15000 non-null  float64
 15  f15     15000 non-null  float64
 16  f16     15000 non-null  float64
 17  f17     15000 non-null  float64
 18  f18     15000 non-null  float64
 19  f19     15000 non-null  float64
 20  f20     15000 non-null

/tmp/ipython-input-3206880668.py:13: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(x='target', data=df, palette='viridis')


### 2. Определение X и y, Train/Test сплит

In [25]:
# Удаляем id, так как это технический столбец, не несущий предсказательной силы
X = df.drop(['id', 'target'], axis=1)
y = df['target']

# random_state для воспроизводимости, stratify для сохранения пропорций классов
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


### 3. Инициализация хранилищ результатов и функция логирования метрик

In [26]:
metrics_test = []
search_summaries = {}

def log_metrics(model, name, X_tst, y_tst):
    y_pred = model.predict(X_tst)
    y_proba = model.predict_proba(X_tst)

    acc = accuracy_score(y_tst, y_pred)
    f1 = f1_score(y_tst, y_pred, average='macro')
    # Для мультикласса используем One-vs-Rest ROC-AUC
    auc = roc_auc_score(y_tst, y_proba, multi_class='ovr', average='macro')

    res = {
        "model": name,
        "accuracy": round(acc, 4),
        "f1_macro": round(f1, 4),
        "roc_auc_ovr": round(auc, 4)
    }
    metrics_test.append(res)
    return res


### 4. Baseline модели

In [27]:
# Dummy
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(X_train, y_train)
log_metrics(dummy, "Dummy", X_test, y_test)

# Logistic Regression (с масштабированием)
lr_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(random_state=42, max_iter=1000))
])
lr_pipe.fit(X_train, y_train)
log_metrics(lr_pipe, "LogisticRegression", X_test, y_test)


{'model': 'LogisticRegression',
 'accuracy': 0.7237,
 'f1_macro': 0.6651,
 'roc_auc_ovr': np.float64(0.8481)}

### 5. Модели Недели 6 (с CV подбором)

#### Decision Tree

In [28]:
dt_params = {'max_depth': [3, 5, 10], 'min_samples_leaf': [1, 5, 10]}
dt_grid = GridSearchCV(DecisionTreeClassifier(random_state=42), dt_params, cv=5, scoring='f1_macro')
dt_grid.fit(X_train, y_train)
search_summaries["DecisionTree"] = {"params": dt_grid.best_params_, "cv_f1": dt_grid.best_score_}
log_metrics(dt_grid.best_estimator_, "DecisionTree", X_test, y_test)


{'model': 'DecisionTree',
 'accuracy': 0.7943,
 'f1_macro': 0.7385,
 'roc_auc_ovr': np.float64(0.8597)}

#### Random Forest

In [29]:
rf_params = {'n_estimators': [50, 100], 'max_depth': [5, 10], 'max_features': ['sqrt', 'log2']}
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=5, scoring='f1_macro')
rf_grid.fit(X_train, y_train)
search_summaries["RandomForest"] = {"params": rf_grid.best_params_, "cv_f1": rf_grid.best_score_}
log_metrics(rf_grid.best_estimator_, "RandomForest", X_test, y_test)

{'model': 'RandomForest',
 'accuracy': 0.8587,
 'f1_macro': 0.8174,
 'roc_auc_ovr': np.float64(0.9447)}

In [30]:
# 2.3.4. Gradient Boosting (один из вариантов бустинга)
#gb_params = {
#    'n_estimators': [50, 100],
#    'learning_rate': [0.01, 0.1],
#    'max_depth': [3, 5]
#}
#gb_grid = GridSearchCV(
#    GradientBoostingClassifier(random_state=42),
#    gb_params,
#    cv=5,
#    scoring='f1_macro'
#)
#gb_grid.fit(X_train, y_train)
#search_summaries["GradientBoosting"] = {
#    "params": gb_grid.best_params_,
#    "cv_f1": gb_grid.best_score_
#}
#log_metrics(gb_grid.best_estimator_, "GradientBoosting", X_test, y_test)

In [31]:
# HistGradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

# Упрощаем параметры для скорости
gb_params = {
    'max_iter': [50, 100],  # аналог n_estimators
    'learning_rate': [0.1, 0.3],
    'max_depth': [3, 5],
    'l2_regularization': [0, 0.1]  # регуляризация для скорости
}

gb_grid = GridSearchCV(
    HistGradientBoostingClassifier(
        random_state=42,
        max_bins=128,  # уменьшаем для скорости
        early_stopping=True,  # ранняя остановка
        n_iter_no_change=5  # останавливаемся если нет улучшений 5 итераций
    ),
    gb_params,
    cv=3,  # уменьшаем CV с 5 до 3
    scoring='f1_macro',
    n_jobs=-1  # используем все ядра процессора
)

print("Начинаем обучение HistGradientBoostingClassifier...")
gb_grid.fit(X_train, y_train)
print("Обучение завершено!")

search_summaries["HistGradientBoosting"] = {
    "params": gb_grid.best_params_,
    "cv_f1": gb_grid.best_score_
}
log_metrics(gb_grid.best_estimator_, "HistGradientBoosting", X_test, y_test)

Начинаем обучение HistGradientBoostingClassifier...
Обучение завершено!


{'model': 'HistGradientBoosting',
 'accuracy': 0.8877,
 'f1_macro': 0.8616,
 'roc_auc_ovr': np.float64(0.9543)}

In [32]:
# 2.3.6. Выбор лучшей модели и интерпретация
# Определяем лучшую модель по ROC-AUC OVR (для мультикласса)
best_model_info = max(metrics_test, key=lambda x: x['roc_auc_ovr'])
print(f"\n--- Лучшая модель ---")
print(f"Модель: {best_model_info['model']}")
print(f"ROC-AUC OVR: {best_model_info['roc_auc_ovr']}")
print(f"F1-macro: {best_model_info['f1_macro']}")


--- Лучшая модель ---
Модель: HistGradientBoosting
ROC-AUC OVR: 0.9543
F1-macro: 0.8616


In [34]:
# Загружаем лучшую модель
best_models = {
    "DecisionTree": dt_grid.best_estimator_,
    "RandomForest": rf_grid.best_estimator_,
    "HistGradientBoosting": gb_grid.best_estimator_,
    "LogisticRegression": lr_pipe,
    "Dummy": dummy
}
best_model = best_models[best_model_info['model']]

In [35]:
# После выбора лучшей модели добавьте:

# Permutation Importance для лучшей модели
print("\n--- Permutation Importance (топ-15 признаков) ---")
perm_importance = permutation_importance(
    best_model, X_test, y_test,
    n_repeats=10,
    random_state=42,
    scoring='f1_macro'
)

# Создаем DataFrame с важностью признаков
importance_df = pd.DataFrame({
    'feature': X.columns,
    'importance_mean': perm_importance.importances_mean,
    'importance_std': perm_importance.importances_std
}).sort_values('importance_mean', ascending=False)

print(importance_df.head(15))

# Визуализация важности признаков
plt.figure(figsize=(12, 8))
top_features = importance_df.head(15)
plt.barh(range(len(top_features)), top_features['importance_mean'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Permutation Importance')
plt.title(f'Top-15 важных признаков для {best_model_info["model"]}')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('artifacts/figures/feature_importance.png', dpi=300, bbox_inches='tight')
plt.close()

# Confusion Matrix для лучшей модели
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_pred_best = best_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred_best)

plt.figure(figsize=(10, 8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=best_model.classes_)
disp.plot(cmap='Blues')
plt.title(f'Confusion Matrix: {best_model_info["model"]}')
plt.savefig('artifacts/figures/confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.close()


--- Permutation Importance (топ-15 признаков) ---
   feature  importance_mean  importance_std
12     f13         0.133301        0.006522
27     f28         0.098848        0.005618
4      f05         0.073389        0.006374
14     f15         0.057960        0.002765
11     f12         0.056605        0.004472
9      f10         0.054843        0.003352
0      f01         0.049915        0.004055
16     f17         0.048403        0.005111
6      f07         0.033673        0.002558
5      f06         0.031436        0.003305
17     f18         0.030425        0.002325
10     f11         0.022399        0.004022
26     f27         0.018709        0.004107
21     f22         0.012176        0.002187
2      f03         0.004572        0.002267


<Figure size 1000x800 with 0 Axes>

In [36]:
import json

# Сохранение метрик
with open('artifacts/metrics_test.json', 'w', encoding='utf-8') as f:
    json.dump(metrics_test, f, indent=2, ensure_ascii=False)

# Сохранение результатов поиска
with open('artifacts/search_summaries.json', 'w', encoding='utf-8') as f:
    json.dump(search_summaries, f, indent=2, ensure_ascii=False)

# Сохранение лучшей модели
joblib.dump(best_model, 'artifacts/best_model.joblib')

# Метаданные лучшей модели
best_model_meta = {
    "best_model": best_model_info['model'],
    "best_params": search_summaries.get(best_model_info['model'], {}).get("params", "default"),
    "test_metrics": best_model_info
}

with open('artifacts/best_model_meta.json', 'w', encoding='utf-8') as f:
    json.dump(best_model_meta, f, indent=2, ensure_ascii=False)

# Дополнительная визуализация: сравнение метрик
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

models = [m['model'] for m in metrics_test]
accuracy = [m['accuracy'] for m in metrics_test]
f1 = [m['f1_macro'] for m in metrics_test]
roc_auc = [m['roc_auc_ovr'] for m in metrics_test]

axes[0].bar(models, accuracy)
axes[0].set_title('Accuracy на тестовой выборке')
axes[0].set_ylabel('Accuracy')
axes[0].tick_params(axis='x', rotation=45)

axes[1].bar(models, f1)
axes[1].set_title('F1-macro на тестовой выборке')
axes[1].set_ylabel('F1-macro')
axes[1].tick_params(axis='x', rotation=45)

axes[2].bar(models, roc_auc)
axes[2].set_title('ROC-AUC OVR на тестовой выборке')
axes[2].set_ylabel('ROC-AUC OVR')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('artifacts/figures/metrics_comparison.png', dpi=300, bbox_inches='tight')
plt.close()